In [73]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.stats as stats
import copy
import os

#for importing data
import torchvision
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Subset
import pandas as pd

# To read an image from url
from imageio import imread

#for data visualization
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

C:\Users\elmog\AppData\Local\Temp\ipykernel_25384\1696291578.py:23: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


In [74]:
#GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [75]:
os.environ['TORCH_HOME'] = './'

In [76]:
# Import vgg Model
trainedModel = torchvision.models.vgg19(pretrained=True)

c:\Users\elmog\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\elmog\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\elmog\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\hub.py:739: UserWarning: TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead
  warnings.warn('TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead')


In [78]:
# Freeze all layers
# for p in trainedModel.named_parameters():
    # p[1].requires_grad = False
    
# Freeze all layers
for p in trainedModel.parameters():
   p.requires_grad = False

In [79]:
# Set to evaluation mode
trainedModel.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [80]:
# use GPU
trainedModel.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [81]:
# Import content image
contentImage = imread("./images/content-image.jpg")

# Import style image
styleImage = imread("./images/style-image1.jpg")
# styleImage = imread("./image/style-image2.jpg")

# Target image
targetImage = np.random.randint(low=0, high=255, size=contentImage.shape, dtype=np.uint8)

print(contentImage.shape)
print(styleImage.shape)
print(targetImage.shape)

C:\Users\elmog\AppData\Local\Temp\ipykernel_25384\3235472342.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  contentImage = imread("./images/content-image.jpg")
C:\Users\elmog\AppData\Local\Temp\ipykernel_25384\3235472342.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  styleImage = imread("./images/style-image1.jpg")


(2084, 4624, 3)
(2084, 4624, 3)
(2084, 4624, 3)


In [82]:
# Transformations
# Step 0 : normalize
# Step 1 : convert to tensor
transform = T.Compose([
    T.ToTensor(),
    T.Resize(256),
    T.Normalize( #use recommended vgg mean & std dev for normalization
        mean=[0.485,0.456,0.406],
        std =[0.229,0.224,0.225]
        )
    #T.RandomHorizontalFlip(p=0.5)
])

In [83]:
# apply transformations + unsqueeze => to add 0 to act as a 4th dimension "4D tensor"
#Model Input Expectation: Most models like VGG (used in style transfer) expect a batch of images as input. This means the input should have the shape [B, C, H, W], where:
#B (Batch size): The number of images being processed simultaneously. A batch size of 1 means you are processing just one image.
#C (Channels), H (Height), and W (Width) 
# This changes the shape of the image tensor from [C, H, W] to [1, C, H, W], making it a 4D tensor. 
# This is necessary because the model expects the input to have the shape of a batch of images, even if it's just a single image.

contentImage= transform( contentImage ).unsqueeze(0).to(device)
styleImage  = transform(  styleImage  ).unsqueeze(0).to(device)
targetImage = transform( targetImage  ).unsqueeze(0).to(device)

print(contentImage.shape)
print(styleImage.shape)
print(targetImage.shape)

torch.Size([1, 3, 256, 568])
torch.Size([1, 3, 256, 568])
torch.Size([1, 3, 256, 568])


c:\Users\elmog\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [84]:
# check images BEFORE style transfer

xViewLength = 8
fig,ax = plt.subplots(3,1, figsize=(xViewLength,xViewLength*568/256))

# matplot lib uses (Width, Height, channels)
img = contentImage.cpu().squeeze().numpy().transpose((1,2,0))
img = (img - np.min(img)) / (np.max(img) - np.min(img))
ax[0].imshow(img)
ax[0].set_title('Content Image')

img = styleImage.cpu().squeeze().numpy().transpose((1,2,0))
img = (img - np.min(img)) / (np.max(img) - np.min(img))
ax[1].imshow(img)
ax[1].set_title('Style Image')

img = targetImage.cpu().squeeze().numpy().transpose((1,2,0))
img = (img - np.min(img)) / (np.max(img) - np.min(img))
ax[2].imshow(img)
ax[2].set_title('Target Image')

Text(0.5, 1.0, 'Target Image')

In [85]:
def modelLayersFeatureMaps(img, trainedModel):
    
    # Initialize feature maps as a list
    featureMaps     = []
    featureNames    = []
    
    convLayerIdx    = 0
    
    # loop through all layers in "features" block
    for layer in range(len(trainedModel.features)):

        # print out info from this layer
        print(layer, trainedModel.features[layer])
        
        # process the image through this layer
        img = trainedModel.features[layer](img)
        
        # store the img if it's a conv2d layer
        if 'Conv2d' in str(trainedModel.features[layer]):
            featureMaps.append( img)
            featureNames.append('ConvLayer_' + str(convLayerIdx))
            convLayerIdx += 1
        
        return featureMaps, featureNames

In [86]:
def gram_matrix(matrix):
    
    # step1: get matrix channels, height, width
    _,channels,height,width = matrix.shape
    
    # step2: reshape matrix to 2D
    matrix = matrix.reshape(channels, width * height)
    
    # Compute & return covariance matrix divided by total number of pixels in that map or image
    gram = torch.mm(matrix,matrix.t()) / (channels * width * height)
    
    return gram

In [89]:
#get the feature maps & feature names list
featureMaps,featureNames = modelLayersFeatureMaps(contentImage, trainedModel)

for i in range(len(featureNames)):
    print(f'length of featureNames = {len(featureNames)}')
    print('feature map "%s" is size %s'%(featureNames[i],(featureMaps[i].shape)))

0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
length of featureName1
feature map "ConvLayer_0" is size torch.Size([1, 64, 256, 568])
